In [38]:
import pandas as pd
import numpy as np

from tabulate import tabulate

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

Load source data from Github

In [39]:
url = 'https://raw.githubusercontent.com/vicky727luo/Maxlulu/main/Lending%20Club%20Project/loan_training_data.csv'
TrainingData = pd.read_csv(url)
TrainingData.head()

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,purpose,addr_state,dti,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,loan_status
0,1077501,5000,5000,4975.0,36 months,10.65,162.87,B,10+ years,RENT,24000.0,Verified,credit_card,AZ,27.65,31048,1,3,0,13648,83.7,9,0,0,5861.071414,5831.78,5000.00,861.07,Fully Paid
1,1077430,2500,2500,2500.0,60 months,15.27,59.83,C,Less than 1 year,RENT,30000.0,Source Verified,car,GA,1.00,36251,5,3,0,1687,9.4,4,0,0,1008.710000,1008.71,456.46,435.17,Charged Off
2,1077175,2400,2400,2400.0,36 months,15.96,84.33,C,10+ years,RENT,12252.0,Not Verified,small_business,IL,8.72,43040,2,2,0,2956,98.5,10,0,0,3003.653644,3003.65,2400.00,603.65,Fully Paid
3,1076863,10000,10000,10000.0,36 months,13.49,339.31,C,10+ years,RENT,49200.0,Source Verified,other,CA,20.00,35096,1,10,0,5598,21.0,37,0,0,12226.302210,12226.30,10000.00,2209.33,Fully Paid
4,1075269,5000,5000,5000.0,36 months,7.90,156.46,A,3 years,RENT,36000.0,Source Verified,wedding,AZ,11.20,43043,3,9,0,7963,28.3,12,0,0,5631.377753,5631.38,5000.00,631.38,Fully Paid


Data Cleaning/Integration

In [40]:
# map column value with index
mapping = {label:idx for idx, label in enumerate(np.unique(TrainingData['term']))}
TrainingData['term']=TrainingData['term'].map(mapping)
mapping = {label:idx for idx, label in enumerate(np.unique(TrainingData['grade']))}
TrainingData['grade']=TrainingData['grade'].map(mapping)
mapping = {label:idx for idx, label in enumerate(np.unique(TrainingData['home_ownership']))}
TrainingData['home_ownership']=TrainingData['home_ownership'].map(mapping)

mapping = {label:idx for idx, label in enumerate(np.unique(TrainingData['verification_status']))}
TrainingData['verification_status']=TrainingData['verification_status'].map(mapping)

mapping = {label:idx for idx, label in enumerate(np.unique(TrainingData['purpose']))}
TrainingData['purpose']=TrainingData['purpose'].map(mapping)

mapping = {label:idx for idx, label in enumerate(np.unique(TrainingData['addr_state']))}
TrainingData['addr_state']=TrainingData['addr_state'].map(mapping)

mapping = {label:idx for idx, label in enumerate(np.unique(TrainingData['loan_status']))}
TrainingData['loan_status']=TrainingData['loan_status'].map(mapping)

TrainingData = TrainingData.select_dtypes(include=[np.number]).interpolate().dropna()
TrainingData = TrainingData.drop(["total_pymnt"], axis=1)
TrainingData = TrainingData.drop(["total_pymnt_inv"], axis=1)
TrainingData = TrainingData.drop(["total_rec_int"], axis=1)

TrainingData.head()

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,home_ownership,annual_inc,verification_status,purpose,addr_state,dti,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_rec_prncp,loan_status
0,1077501,5000,5000,4975.0,0,10.65,162.87,1,2,24000.0,2,1,3,27.65,31048,1,3,0,13648,83.7,9,0,0,5000.00,1
1,1077430,2500,2500,2500.0,1,15.27,59.83,2,2,30000.0,1,0,10,1.00,36251,5,3,0,1687,9.4,4,0,0,456.46,0
2,1077175,2400,2400,2400.0,0,15.96,84.33,2,2,12252.0,0,10,12,8.72,43040,2,2,0,2956,98.5,10,0,0,2400.00,1
3,1076863,10000,10000,10000.0,0,13.49,339.31,2,2,49200.0,1,8,4,20.00,35096,1,10,0,5598,21.0,37,0,0,10000.00,1
4,1075269,5000,5000,5000.0,0,7.90,156.46,0,2,36000.0,1,12,3,11.20,43043,3,9,0,7963,28.3,12,0,0,5000.00,1


Train-test split

In [41]:
yPredict = TrainingData.loan_status
XClean = TrainingData.drop(["loan_status"], axis=1)

X_train, X_test, y_train, y_test = train_test_split(XClean, yPredict, random_state=42, test_size=.30)

In [42]:
yPredict.head()

0    1
1    0
2    1
3    1
4    1
Name: loan_status, dtype: int64

Transform File

In [43]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

Nerual Network

In [44]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(30,30,30))
mlp.fit(X_train,y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(30, 30, 30), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

Prediction

In [45]:
predictions = mlp.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,predictions))

[[ 438   18]
 [   1 2245]]


Evaluation

In [46]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       1.00      0.96      0.98       456
           1       0.99      1.00      1.00      2246

    accuracy                           0.99      2702
   macro avg       0.99      0.98      0.99      2702
weighted avg       0.99      0.99      0.99      2702

